In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import os
import pandas as pd
from tqdm import tqdm
import torch

In [ ]:
from video_loading import balance_videos_list
from extract_keypoints import KeypointInterpolator

folder_path = r'10 class 28 actor (different size)'
num_labels = balance_videos_list(folder_path)

interpolator = KeypointInterpolator(num_labels, frames=80)
interpolator.run()

a = np.load(f'vsl{num_labels}_data_preprocess.npy')
b = np.load(f'vsl{num_labels}_label_preprocess.npy')

print(a.shape)
print(b.shape)

In [ ]:
def k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds, destination_folder="numpy_files"):
    os.makedirs(destination_folder, exist_ok=True)

    actors = train_data['actor'].unique()
    print(f"Number of actors: {len(actors)}")
    print('-----------------------------------------------------')

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    actor_to_indices = {actor: train_data.index[train_data['actor'] == actor].tolist() for actor in actors}
    folds = [[] for _ in range(k_folds)]

    for fold, (train_actors, test_actors) in enumerate(kf.split(actors)):
        train_actors = actors[train_actors]
        test_actors = actors[test_actors]

        for actor in test_actors:
            folds[fold].extend(actor_to_indices[actor])

        tqdm.write(f"Fold {fold+1}: {len(folds[fold])} test samples")

    # Iterate over each fold to create train-test splits
    for fold in range(k_folds):
        test_indices = folds[fold]
        train_indices = [idx for f in range(k_folds) if f != fold for idx in folds[f]]

        X_train, X_test = keypoint_data[train_indices], keypoint_data[test_indices]
        y_train = np.array(label_data[train_indices], dtype=np.int64)
        y_test = np.array(label_data[test_indices], dtype=np.int64)

        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_train.npy'), X_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_train.npy'), y_train)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_data_fold{fold+1}_test.npy'), X_test)
        np.save(os.path.join(destination_folder, f'vsl{num_labels}_label_fold{fold+1}_test.npy'), y_test)

        tqdm.write(f"Processed and saved vsl{num_labels} fold {fold+1} successfully.")

if __name__ == "__main__":
    input_file_path = f"vsl{num_labels}_interpolated_keypoints.csv"
    train_data = pd.read_csv(input_file_path)

    keypoint_data = np.load(f'vsl{num_labels}_data_preprocess.npy')
    label_data = np.load(f'vsl{num_labels}_label_preprocess.npy')

    num_labels = len(np.unique(label_data))

    k_folds = 10
    k_fold_cross_validation(train_data, keypoint_data, label_data, num_labels, k_folds)


In [ ]:
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from feeder import FeederINCLUDE
from aagcn import Model
from augumentation import Rotate, Compose
from pytorch_lightning.utilities.migration import pl_legacy_patch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

if __name__ == '__main__':
    k_folds = 10
    config = {'batch_size': 128, 'learning_rate': 0.0137296, 'weight_decay': 0.000150403}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    best_accuracy = 0.0
    best_fold = -1

    for fold in range(k_folds):
        print(f"Starting fold {fold + 1}/{k_folds}")
        train_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_train.npy')
        train_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_train.npy')
        val_data_path = os.path.join("numpy_files", f'vsl{num_labels}_data_fold{fold+1}_test.npy')
        val_label_path = os.path.join("numpy_files", f'vsl{num_labels}_label_fold{fold+1}_test.npy')

        transforms = Compose([
            Rotate(15, 80, 25, (0.5, 0.5))
        ])

        train_dataset = FeederINCLUDE(
            data_path=train_data_path,
            label_path=train_label_path,
            transform=transforms
        )
        val_dataset = FeederINCLUDE(
            data_path=val_data_path,
            label_path=val_label_path
        )

        train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

        model = Model(num_class=num_labels, num_point=46, num_person=1, in_channels=2,
                      graph_args={"layout": "mediapipe_two_hand", "strategy": "spatial"},
                      learning_rate=config['learning_rate'], weight_decay=config['weight_decay'])

        # Path pre-trained checkpoint file
        checkpoint_path = "autsl_vsl199-aagcn-fold=7-v1.ckpt"

        with pl_legacy_patch():
            checkpoint = torch.load(checkpoint_path, map_location=device)

        state_dict = checkpoint['state_dict']
        filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('fc.')}
        model.load_state_dict(filtered_state_dict, strict=False)

        callbacks = [
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="valid_accuracy",
                mode="max",
                every_n_epochs=1,
                filename=f'autsl_vsl{num_labels}-aagcn-fold={fold+1}'
            ),
        ]

        trainer = pl.Trainer(max_epochs=100, accelerator="auto", check_val_every_n_epoch=1,
                             devices=1, callbacks=callbacks)

        trainer.fit(model, train_dataloader, val_dataloader)
        val_accuracy = trainer.callback_metrics['valid_accuracy'].item()
        print(f"Fold {fold + 1} finished with validation accuracy: {val_accuracy:.4f}")

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_fold = fold + 1

    print(f"The highest validation accuracy achieved is {best_accuracy:.4f} from fold {best_fold}.")


In [ ]:
print(f"The highest validation accuracy achieved of autsl vsl{num_labels} is {best_accuracy:.4f} from fold {best_fold}.")